In [1]:
import pandas as pd
from tbparse import SummaryReader
import os
from torch.utils.tensorboard import SummaryWriter

2023-05-01 03:51:20.587263: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-05-01 03:51:20.607639: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
def combine_tensorboard_graphs(log_dir, experiment_name):
    text_tag = "config"
    dataFrames = []
    dirs = sorted(os.listdir(f"{log_dir}/{experiment_name}"))
    logs = []
    for dir in dirs:
        if dir[:5] == "logs-":
            logs.append(dir)
    writer = SummaryWriter(log_dir=f"{log_dir}/{experiment_name}/{logs[-1]}_combined")
    for dir in logs:
        reader = SummaryReader(f"{log_dir}/{experiment_name}/{dir}")
        dataFrames.append(reader.scalars)

    final_dt = dataFrames[0]
    scallar_tags = final_dt.tag.unique().tolist()

    text = (reader.text)["value"].values[0]

    writer.add_text(tag=text_tag, text_string=text)
    for tag in scallar_tags:
        tag_dt = final_dt[final_dt["tag"] == tag]
        last_entry = tag_dt.tail(1)["step"].index.values[0]
        for dt in dataFrames[1:]:
            new_entries = dt[dt.tag == tag]
            tag_dt = pd.concat([tag_dt.iloc[:last_entry],new_entries,tag_dt.iloc[last_entry:]]).reset_index(drop=True)
            tag_dt = tag_dt.drop_duplicates(subset='step', keep='first')
            tag_dt = tag_dt.sort_values("step").reset_index(drop=True)
        for idx in range(len(tag_dt)):
            entry = tag_dt.iloc[idx]
            writer.add_scalar(tag=entry["tag"], scalar_value=entry["value"], global_step=entry["step"])
    writer.close()

In [3]:
log_dir = "/home/manos/Thesis/opendr/projects/python/perception/object_detection_2d/nanodet/temp"

experiments = ["augmented_data", "augmented_data_with_augmentations", "norm_data_with_augmentations"]
for experiment_name in experiments:
    combine_tensorboard_graphs(log_dir, experiment_name)